### get raw data

In [ ]:
# Set workspace to Windrecorder dir
import os
os.chdir("..")
os.chdir("..")
# ------------------------------------------------------------

import datetime
import pandas as pd

from windrecorder.db_manager import db_manager
from windrecorder.oneday import OneDay

In [ ]:
df_day_search_result = OneDay().search_day_data(
                    datetime.datetime.today(),
                    search_content="",
                )

df_day_search_result

### clean and format data

In [ ]:
df_day_search_result['videofile_time'] = pd.to_datetime(df_day_search_result['videofile_time'], unit='s').dt.strftime('%Y-%m-%d %H-%M-%S')

clean_df = pd.DataFrame(columns=['datetime', 'activity', 'ocr_content'])
activity_compare_lst = []
max_token_limit = 1000000/2.5
max_pre_row_token = int(max_token_limit/len(df_day_search_result))
activity_deduplication_trace_depth = 10

for index, row in df_day_search_result.iterrows():
    activity = row['win_title'].split(" | ")[0]
    if activity in activity_compare_lst[-activity_deduplication_trace_depth:] or len(activity) == 0:
        continue
    activity_compare_lst.append(activity)
    ocr_text = row['ocr_text']
    
    row = {
        'datetime': row['videofile_time'],
        'activity': activity,
        'ocr_content': row['ocr_text'][:max_pre_row_token],
    }
    clean_df.loc[len(clean_df )] = row

clean_df